Consider different types of group assignments, and measure the within-group consistency (pairwise distances).

Start with the one currently implemented in `group_matching.py`

In [1]:
import pandas as pd

import numpy as np
import pandas as pd
from tqdm import tqdm

from scipy.cluster.hierarchy import linkage
import hcluster   # requires dedupe-hcluster
from paper_reviewer_matcher import (
    preprocess, compute_affinity
)

from group_matching import compute_conflicts, generate_pod_numbers

users = pd.read_csv('data/mindmatch_example.csv').to_dict(orient='records')
n_users = len(users)
print('Number of registered users: {}'.format(n_users))

users_df = pd.DataFrame(users).fillna('')
users_dict = {r['user_id']: dict(r) for _, r in users_df.iterrows()}  # map of user id to details
persons_1 = list(map(preprocess, list(users_df['abstracts'])))
persons_2 = list(map(preprocess, list(users_df['abstracts'])))
A = compute_affinity(
    persons_1, persons_2,
    n_components=30, min_df=2, max_df=0.8,
    weighting='tfidf', projection='svd'
)
cois_list = compute_conflicts(users_df)
for i, j in cois_list:
    A[i, j] = -1

A_cluster = - A
A_cluster[A_cluster == 1000] = 1
A_rand = np.random.randn(n_users, n_users) * 0.01 * A_cluster.var() # add randomness

z = linkage(A_cluster + A_rand,
            method='average',
            metric='euclidean',
            optimal_ordering=True)
cluster = hcluster.fcluster(z, t=0.01,
                            criterion='distance') # distance

# Measure the goodness of the assignment
def generate_pod_numbers(n_users, n_per_group):
    """
    Generate pod numbers in sequence
    """
    ngroups = int(np.ceil(A.shape[0] / n_per_group))
    nsmallgroups = ngroups * n_per_group - A.shape[0]
    nbiggroups = ngroups - nsmallgroups
    
    group_sizes = [n_per_group] * nbiggroups + [n_per_group - 1] * nsmallgroups
    groups = []
    for i, gs in enumerate(group_sizes):
        groups.extend([i] * gs)
    return groups

cluster_numbers = generate_pod_numbers(n_users=A.shape[0], n_per_group=5)

Using Google ortools library for ILP solver.
Number of registered users: 1162


/Users/patrickmineault/miniconda3/envs/py3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
1162it [02:56,  6.59it/s]


In [2]:
def measure_goodness(A_cluster, cluster_assignments):
    dists = []
    for i in range(cluster_assignments.min(), cluster_assignments.max()+1):
        # Calculate the average pairwise distance within the cluster.
        mean_dist = A_cluster[cluster_assignments == i, :][:, cluster_assignments == i].mean()
        dists.append(mean_dist)
        
    return dists

assignments = sorted([(id_, cluster_numbers[cluster]) for cluster, id_ in sorted(zip(cluster, np.arange(A.shape[0])))])
assignments = np.array([y for x, y in assignments])

# Measure the conventional method
print("Conventional method")
goodnesses = np.array(measure_goodness(A, assignments))
print([goodnesses.mean(), np.std(goodnesses)])

print("Random assignment")
goodnesses = np.array(measure_goodness(A, np.array(cluster_numbers)))
print([goodnesses.mean(), np.std(goodnesses)])


Conventional method
[-0.23604346738459883, 0.06337616032926523]
Random assignment
[-0.3248631461787265, 0.04439390372145903]


Now implement an alternative method based off of multi-pass hierarchical clustering

In [3]:
# We apply the alternative bottom-up method suggested here
# https://github.com/jmonlong/Hippocamplus/blob/master/content/post/2018-06-09-ClusterEqualSize.Rmd
def agglomerate(A, group_size):
    ngroups = int(np.ceil(A.shape[0] / group_size))
    nsmallgroups = ngroups * group_size - A.shape[0]
    nbiggroups = ngroups - nsmallgroups
    labels = np.ones(A.shape[0]) * np.nan

    A0 = A.copy()
    
    groups = []
    group_sizes = [group_size] * nbiggroups + [group_size - 1] * nsmallgroups
    assert A.shape[0] == sum(group_sizes)
    j = 0
    for gs in tqdm(group_sizes):
        B = A[np.isnan(labels), :][:, np.isnan(labels)]
        z = linkage(B,
                    method='average',
                    metric='euclidean')
        
        the_nums = np.where(z[:, -1] >= gs)[0]
        if the_nums.size == 0:
            print(the_nums)
            print(z)
            print(B.shape)
        minpos = the_nums.min()
        
        cluster_nums = [z[minpos, 0], z[minpos, 1]]
        
        i = 0
        while i < len(cluster_nums):
            if cluster_nums[i] >= B.shape[0]:
                cluster_nums.append(z[int(cluster_nums[i]) - B.shape[0], 0])
                cluster_nums.append(z[int(cluster_nums[i]) - B.shape[0], 1])
            i += 1
            
        cluster_nums = np.array(cluster_nums).astype(int)
        cluster_nums = cluster_nums[cluster_nums < B.shape[0]]
        
        assert len(cluster_nums) >= gs
        cluster_nums = cluster_nums[:gs]
        
        # Map cluster nums to the original numbers prior to subsetting.
        the_map = np.where(np.isnan(labels))[0]        
        cluster_nums = [the_map[k] for k in cluster_nums]
        labels[cluster_nums] = j        
        j += 1
        
    return labels.astype(int)

labels = agglomerate(A_cluster + A_rand, 5)

print("Bottom up")
goodnesses = np.array(measure_goodness(A, labels))
print([goodnesses.mean(), np.std(goodnesses)])

100%|██████████████████████████████████████████████████████████████████████████████████████| 233/233 [00:14<00:00, 16.17it/s]

Bottom up
[-0.22207694200864384, 0.057677851594478996]


The second method does a little bit better.